In [ ]:
class CSP: #Given question.
    """A simple representation of a Constraint Satisfaction Problem."""

    def __init__(self, variables, domains, neighbors, constraints):
        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraints = constraints

    def assign(self, var, val, assignment):
        """Assign a value to a variable."""
        assignment[var] = val

    def nconflicts(self, var, val, assignment):
        """Count conflicts with other variables."""
        return sum(1 for neighbor in self.neighbors[var]
                   if neighbor in assignment and not self.constraints(var, val, neighbor, assignment[neighbor]))

    def goal_test(self, assignment):
        """Check if all variables are assigned."""
        return len(assignment) == len(self.variables)

# Example usage
def different_values_constraint(A, a, B, b):
    """A constraint saying two neighboring variables must differ in value."""
    return a != b

colors = 'RGB'
variables = 'ABC'
domains = {var: list(colors) for var in variables}
neighbors = {'A': ['B', 'C'], 'B': ['A', 'C'], 'C': ['A', 'B']}

csp = CSP(variables, domains, neighbors, different_values_constraint)

# Example of a simple backtracking search
def backtracking_search(csp):
    """A simple backtracking search algorithm."""
    def backtrack(assignment):
        if csp.goal_test(assignment):
            return assignment
        var = next(v for v in csp.variables if v not in assignment)
        for value in csp.domains[var]:
            if csp.nconflicts(var, value, assignment) == 0:
                csp.assign(var, value, assignment)
                result = backtrack(assignment)
                if result is not None:
                    return result
                del assignment[var]  # Unassign if it didn't lead to a solution
        return None

    return backtrack({})

solution = backtracking_search(csp)
if solution:
    print("Solution found:", solution)
else:
    print("No solution found.")

Solution found: {'A': 'R', 'B': 'G', 'C': 'B'}


In [ ]:
#modified code with forward-checking: (answer)
# Backtracking search with forward checking
class CSP:
    """A simple representation of a Constraint Satisfaction Problem."""

    def __init__(self, variables, domains, neighbors, constraints):
        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraints = constraints

    def assign(self, var, val, assignment):
        """Assign a value to a variable."""
        assignment[var] = val

    def nconflicts(self, var, val, assignment):
        """Count conflicts with other variables."""
        return sum(1 for neighbor in self.neighbors[var]
                   if neighbor in assignment and not self.constraints(var, val, neighbor, assignment[neighbor]))

    def goal_test(self, assignment):
        """Check if all variables are assigned."""
        return len(assignment) == len(self.variables)

def different_values_constraint(A, a, B, b):
    """A constraint saying two neighboring variables must differ in value."""
    return a != b

colors = 'RGB'
variables = 'ABC'
domains = {var: list(colors) for var in variables}
neighbors = {'A': ['B', 'C'], 'B': ['A', 'C'], 'C': ['A', 'B']}

csp = CSP(variables, domains, neighbors, different_values_constraint)

# Backtracking search with forward checking
def backtracking_search_fc(csp):
    """A backtracking search algorithm with forward checking."""
    def backtrack(assignment):
        if csp.goal_test(assignment):
            return assignment

        var = next(v for v in csp.variables if v not in assignment)
        domain_copy = {v: list(csp.domains[v]) for v in csp.variables}  # Copy of domains

        for value in csp.domains[var]:
            if csp.nconflicts(var, value, assignment) == 0:
                csp.assign(var, value, assignment)

                # Forward checking
                for neighbor in csp.neighbors[var]:
                    if neighbor not in assignment:
                        csp.domains[neighbor] = [val for val in csp.domains[neighbor]
                                                 if csp.constraints(var, value, neighbor, val)]
                        if not csp.domains[neighbor]:  # If domain is empty, no solution
                            del assignment[var]
                            csp.domains = domain_copy  # Restore domains
                            break
                else:  # Continue backtracking if no domain became empty
                    result = backtrack(assignment)
                    if result is not None:
                        return result
                del assignment[var]  # Unassign if it didn't lead to a solution
                csp.domains = domain_copy  # Restore domains

        return None

    return backtrack({})

solution_fc = backtracking_search_fc(csp)
print("Solution found:", solution_fc)


Solution found: {'A': 'R', 'B': 'G', 'C': 'B'}


Steps involved in solving a CSP (Constraint Satisfaction Problem) are:

Variable Selection: Choose an unassigned variable to assign a value to.

Value Selection: Select a value from the variable's domain.

Constraint Check: Ensure the selected value doesn't violate any constraints with already assigned variables.

Assignment: Assign the value to the variable.

Forward Checking (optional): Update domains of unassigned variables to prune inconsistent values.

Backtracking: If a conflict arises, backtrack to the previous variable and try a different value.

Goal Test: Check if all variables are assigned without conflicts; if so, a solution is found.

In [ ]:
#BOG - LP 11 - Sudoku solver with CSP

import re
from collections import defaultdict

class CSP:
    def __init__(self, variables, domains, neighbors, constraints):
        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraints = constraints

    def assign(self, var, value, assignment):
        """Assign a value to a variable and prune the domain."""
        assignment[var] = value
        self.domains[var] = [value]  # Prune the domain of var to just the assigned value
        for neighbor in self.neighbors[var]:
            if neighbor not in assignment:
                if value in self.domains[neighbor]:
                    self.domains[neighbor].remove(value)

    def unassign(self, var, assignment):
        """Remove a variable assignment and restore its domain."""
        if var in assignment:
            del assignment[var]
            self.domains[var] = list('123456789')  # Restore the domain

    def is_consistent(self, var, value, assignment):
        """Check if the value assignment is consistent with the constraints."""
        return all(self.constraints(var, value, neighbor, assignment[neighbor])
                   for neighbor in self.neighbors[var] if neighbor in assignment)

    def select_unassigned_variable(self, assignment):
        """Select an unassigned variable (MRV heuristic)."""
        return min((v for v in self.variables if v not in assignment), key=lambda v: len(self.domains[v]))

    def order_domain_values(self, var, assignment):
        """Order domain values for the variable (LCV heuristic)."""
        def lcv(value):
            return sum(1 for neighbor in self.neighbors[var] if neighbor not in assignment and value in self.domains[neighbor])
        return sorted(self.domains[var], key=lcv)

    def backtracking_search(self):
        """Perform backtracking search to find a solution."""
        if not self.AC3():  # Ensure initial arc consistency
            return None
        return self.backtrack({})

    def backtrack(self, assignment):
        """Recursive backtracking search algorithm."""
        if len(assignment) == len(self.variables):
            return assignment
        var = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(var, assignment):
            if self.is_consistent(var, value, assignment):
                self.assign(var, value, assignment)
                result = self.backtrack(assignment)
                if result:
                    return result
                self.unassign(var, assignment)
        return None

    def AC3(self):
        """AC3 algorithm for arc consistency."""
        queue = [(xi, xj) for xi in self.variables for xj in self.neighbors[xi]]
        while queue:
            (xi, xj) = queue.pop(0)
            if self.revise(xi, xj):
                if not self.domains[xi]:
                    return False
                for xk in self.neighbors[xi]:
                    if xk != xj:
                        queue.append((xk, xi))
        return True

    def revise(self, xi, xj):
        """Revise the domain of xi based on the constraint with xj."""
        revised = False
        for x in set(self.domains[xi]):
            if all(not self.constraints(xi, x, xj, y) for y in self.domains[xj]):
                self.domains[xi].remove(x)
                revised = True
        return revised


class Sudoku(CSP):
    def __init__(self, grid):
        """Initialize the Sudoku problem from a grid string."""
        rows = 'ABCDEFGHI'
        cols = '123456789'
        self.variables = [r + c for r in rows for c in cols]
        self.domains = {v: list('123456789') for v in self.variables}
        self.neighbors = self.initialize_neighbors()
        CSP.__init__(self, self.variables, self.domains, self.neighbors, self.sudoku_constraints)
        self.parse_grid(grid)

    def parse_grid(self, grid):
        """Convert grid string to dictionary with {variable: value}."""
        values = [c for c in grid if c in '123456789.']
        for i, v in enumerate(self.variables):
            if values[i] in '123456789':
                self.assign(v, values[i], {})  # Assign initial values and prune domains

    def initialize_neighbors(self):
        """Initialize neighbors for each cell in the Sudoku grid."""
        rows = 'ABCDEFGHI'
        cols = '123456789'
        squares = [cross(rows, c) for c in cols] + [cross(r, cols) for r in rows] + [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
        neighbors = defaultdict(set)
        for s in squares:
            for v in s:
                neighbors[v].update(s - {v})
        return neighbors

    def sudoku_constraints(self, var1, val1, var2, val2):
        """Constraint: two variables must have different values."""
        return val1 != val2

    def display(self, assignment):
        """Display the Sudoku grid."""
        rows = 'ABCDEFGHI'
        cols = '123456789'
        width = 2
        line = '+'.join(['-' * (width * 3)] * 3)
        for r in rows:
            print(''.join(assignment.get(r + c, '.').center(width) + ('|' if c in '36' else '') for c in cols))
            if r in 'CF':
                print(line)


def cross(a, b):
    """Cross product of elements in A and elements in B."""
    return {x + y for x in a for y in b}


# Example usage
if __name__ == "__main__":
    easy_sudoku = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
    sudoku = Sudoku(easy_sudoku)
    solution = sudoku.backtracking_search()
    if solution:
        sudoku.display(solution)
    else:
        print("No solution found.")


4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 
